In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row
import timeit

In [2]:
# Importing the required notebooks
%run DifferentialEvolution.ipynb
%run IslandMethod.ipynb

In [ ]:
# Funtion for which to measure time
def wrapper(func, *args, **kwargs):
    def wrapped():
        return func(*args, **kwargs)
    return wrapped

In [ ]:
# Start spark
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('IPDE') \
    .getOrCreate()
sc = spark.sparkContext
        
# Setting the test
rosenberg = lambda x: 100 * (x[1] - x[0] * x[0])**2 + (1 - x[0])**2
diffEvo = DifferentialEvolution(rosenberg, d = 2, bound = 0.1)
island = IslandMethod(sc, diffEvo = diffEvo, localIt = 2)

# Executing the island
wrapped = wrapper(island.execute, sc)
print("Execution time: ", timeit.timeit(wrapped, number = 1), "s", sep = '')